In [9]:
import pandas as pd
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

# Load the file
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename, sep=';', engine='python')

# DK columns: DK01x01 to DK04x04 (total 16 columns)
dk_cols = [f'DK0{i}x{j:02d}' for i in range(1, 5) for j in range(1, 5)]

# Gender columns: SC01_01 to SC01_10
gender_cols = [col for col in df.columns if col.startswith("SC01")]
sexual_orientation_cols = [col for col in df.columns if col.startswith("SC02")]


# Keep only relevant rows with at least one DK response
filtered_df = df[df[dk_cols].notnull().any(axis=1)]

# Function to collect selected identity labels (German: 'ausgewählt' = selected)
def collect_selected(row, columns):
    return [col for col in columns if str(row[col]).strip().lower() == 'ausgewählt']


# Create new columns for selected gender and sexuality
filtered_df['selected_gender'] = filtered_df.apply(lambda row: collect_selected(row, gender_cols), axis=1)
filtered_df['selected_sexuality'] = filtered_df.apply(lambda row: collect_selected(row, sexuality_cols), axis=1)

Saving data_queerbiasKI_2025-05-24_12-00.csv to data_queerbiasKI_2025-05-24_12-00 (3).csv


<ipython-input-9-a4617755f422>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['selected_gender'] = filtered_df.apply(lambda row: collect_selected(row, gender_cols), axis=1)
<ipython-input-9-a4617755f422>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['selected_sexuality'] = filtered_df.apply(lambda row: collect_selected(row, sexuality_cols), axis=1)


In [22]:
import pandas as pd
import csv
with open("data_queerbiasKI_2025-05-24_12-00.csv", encoding="utf-16") as f:
    dialect = csv.Sniffer().sniff(f.read(1024))
    print("Detected delimiter:", dialect.delimiter)
# Load with the first two rows as header information
df_raw = pd.read_csv("data_queerbiasKI_2025-05-24_12-00.csv", header=[0, 1], delimiter=dialect.delimiter, encoding="utf-16")

# Rebuild column names by combining both rows of the header
df_raw.columns = [f"{col[0].strip()}_{col[1].strip()}" if isinstance(col, tuple) else col for col in df_raw.columns]

# Drop any rows that were header rows (e.g. 2nd row repeated)
df = df_raw[2:].copy()

# Define which prefix to look for
gender_cols = [col for col in df.columns if col.startswith('SC01_')]
sexuality_cols = [col for col in df.columns if col.startswith('SC02_')]

# Use second header row as label names (now encoded in column names)
gender_labels = {col: col.split("_", 1)[1] for col in gender_cols}
sexuality_labels = {col: col.split("_", 1)[1] for col in sexuality_cols}

# Function to collect selected identities
def collect_labels(row, label_dict):
    selected = []
    for col, label in label_dict.items():
        val = str(row.get(col, '')).strip().lower()
        if val in ['1', 'ja', 'true', 'yes', 'ausgewählt', 'x']:
            selected.append(label)
    return selected

# Apply to DataFrame
df['gewählte_Geschlechtsidentitäten'] = df.apply(lambda row: collect_labels(row, gender_labels), axis=1)
df['gewählte_sexuelle_Orientierungen'] = df.apply(lambda row: collect_labels(row, sexuality_labels), axis=1)
# Save the cleaned file
output_path = "filtered_identity_data_from_labels.csv"
df.to_csv(output_path, index=False)
print(f"✅ File saved as: {output_path}")


Detected delimiter: 	


KeyError: "['DK01x01', 'DK01x02', 'DK01x03', 'DK01x04', 'DK02x01', 'DK02x02', 'DK02x03', 'DK02x04', 'DK03x01', 'DK03x02', 'DK03x03', 'DK03x04', 'DK04x01', 'DK04x02', 'DK04x03', 'DK04x04'] not in index"

In [23]:
import pandas as pd
import csv

# Load the CSV with multi-row header and UTF-16 encoding, detect delimiter
with open("data_queerbiasKI_2025-05-24_12-00.csv", encoding="utf-16") as f:
    dialect = csv.Sniffer().sniff(f.read(1024))
    print("Detected delimiter:", dialect.delimiter)

df_raw = pd.read_csv("data_queerbiasKI_2025-05-24_12-00.csv", header=[0, 1], delimiter=dialect.delimiter, encoding="utf-16")

# Combine the two header rows to single-level columns
df_raw.columns = [f"{col[0].strip()}_{col[1].strip()}" if isinstance(col, tuple) else col for col in df_raw.columns]

# Drop repeated header rows (e.g. second header row repeated as data)
df = df_raw[2:].copy()

# Define gender and sexuality columns (prefixes)
gender_cols = [col for col in df.columns if col.startswith('SC01_')]
sexuality_cols = [col for col in df.columns if col.startswith('SC02_')]

# Define DK columns by pattern (e.g. DK01x01 to DK04x04)
# If you want, you can build this list dynamically by checking column names containing "DK"
dk_cols = [col for col in df.columns if col.upper().startswith('DK')]

# Use second header row labels (encoded in column names after '_')
gender_labels = {col: col.split("_", 1)[1] for col in gender_cols}
sexuality_labels = {col: col.split("_", 1)[1] for col in sexuality_cols}

# Function to collect selected identities in a row
def collect_labels(row, label_dict):
    selected = []
    for col, label in label_dict.items():
        val = str(row.get(col, '')).strip().lower()
        if val in ['1', 'ja', 'true', 'yes', 'ausgewählt', 'x']:
            selected.append(label)
    return selected

# Collect gender and sexuality labels for each row
df['gender_labels'] = df.apply(lambda row: collect_labels(row, gender_labels), axis=1)
df['sexuality_labels'] = df.apply(lambda row: collect_labels(row, sexuality_labels), axis=1)

# Select only relevant columns: DK + gender_labels + sexuality_labels
df_subset = df[dk_cols + ['gender_labels', 'sexuality_labels']].copy()

# Melt the DK columns into one column with their values
df_melted = df_subset.melt(id_vars=['gender_labels', 'sexuality_labels'], value_vars=dk_cols,
                           var_name='dk_column', value_name='dk_entry')

# Optional: Remove rows where dk_entry is null or empty
df_melted = df_melted[df_melted['dk_entry'].notna() & (df_melted['dk_entry'].astype(str).str.strip() != '')]

# Save to new CSV
output_path = "dk_gender_sexuality_long_format.csv"
df_melted.to_csv(output_path, index=False)

print(f"✅ New CSV saved as: {output_path}")


Detected delimiter: 	
✅ New CSV saved as: dk_gender_sexuality_long_format.csv
